#### Тема: антифрод аналитика 
1. Нужно изучить анонимизированный датасет событий взаимодействия пользователей с рекламными объявлениями
2. выявить в нем аномальные можешь использовать любые Python библиотеки, а также данные из открытых источников (не включащие персональные и другие защищаемые законом данные). 
3. Результат: - отчёт (EDA) о проделанном анализе должен быть представлен в формате .ipynb, с подробными комментариями - алгоритм выделения найденных аномалий в трафике

In [17]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data = pd.read_csv('antifraud_analyst_test.csv')
data.rename(columns = {'Unnamed: 0':'ID'}, inplace = True)
data = data.sort_values('ID')
data = data.drop_duplicates().reset_index(drop=True)
data.head()

,ID,user_id,ip_address_int,click_timestamp,source,sub_source,sub_source_2,sub_source_3,referer,fingerprint,ua
0,0,4391ca69eb0e93c6752125f785fa8ef6,772930238,1635755242,250820642ba26fc02d946cf504440fdf,fba462c8894f1111b7efb1c67a094cf3,7791ccbd2270c2d91531262715bad131,91cc61b3a85e20194659c63d83a68cec,fba462c8894f1111b7efb1c67a094cf3,66b6125631b826ac0f3b4caf308b25ad,axios/0.23.0
1,0,03e014b6b59279eeda2d96a5a3653a2f,1442387889,1635774448,9cbd1454916470e9cefbb56dbb382954,fba462c8894f1111b7efb1c67a094cf3,0ae0c71dc0d7110747b9a13ca44f3f4a,fba462c8894f1111b7efb1c67a094cf3,465d9bf58432d4e1f422b3639caddee8,fba462c8894f1111b7efb1c67a094cf3,NaN
2,0,08f1608c1ff0c09fe0b425448df9ef6a,1348124303,1635796166,9c0c7bc6240316a7cbb1f4c1bcd549d6,fba462c8894f1111b7efb1c67a094cf3,d53eb50aa9e6642204fe1f82a4df29eb,fba462c8894f1111b7efb1c67a094cf3,281228a44f8be3867df599a8f9b93efb,164fa9e24dc44107406e3e3e0bc003bf,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
3,1,4391ca69eb0e93c6752125f785fa8ef6,772930238,1635755242,250820642ba26fc02d946cf504440fdf,fba462c8894f1111b7efb1c67a094cf3,7791ccbd2270c2d91531262715bad131,91cc61b3a85e20194659c63d83a68cec,fba462c8894f1111b7efb1c67a094cf3,66b6125631b826ac0f3b4caf308b25ad,axios/0.23.0
4,1,4d9cc5b94fb63d84aa5fb6afcbfc2835,1546848813,1635776942,1fab684ab69a4a26bae26045e813d602,fba462c8894f1111b7efb1c67a094cf3,35ff6bc1e1b7ef17e5d7730a702df9a9,fba462c8894f1111b7efb1c67a094cf3,600f597c4acb8b3e12e544c732935a68,1d29469882da881efa5c398f64160051,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8633 entries, 5128 to 5082
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               8633 non-null   int64 
 1   user_id          8633 non-null   object
 2   ip_address_int   8633 non-null   int64 
 3   click_timestamp  8633 non-null   int64 
 4   source           8633 non-null   object
 5   sub_source       8633 non-null   object
 6   sub_source_2     8633 non-null   object
 7   sub_source_3     8633 non-null   object
 8   referer          8633 non-null   object
 9   fingerprint      8633 non-null   object
 10  ua               3739 non-null   object
dtypes: int64(3), object(8)
memory usage: 809.3+ KB


In [8]:
data["ID"].value_counts()

0       3
133     3
66      3
74      3
70      3
82      3
115     3
90      3
98      3
78      3
106     3
107     3
114     3
122     3
86      3
130     3
99      3
138     3
146     3
123     3
58      3
62      3
46      3
171     3
22      3
163     3
30      3
155     3
147     3
2       3
10      3
50      3
139     3
18      3
26      3
54      3
34      3
131     3
42      3
94      3
154     3
91      3
190     3
27      3
166     3
19      3
174     3
11      3
182     3
3       3
198     3
35      3
197     3
189     3
181     3
173     3
165     3
157     3
149     3
158     3
150     3
162     3
202     3
170     3
102     3
178     3
83      3
186     3
194     3
110     3
75      3
43      3
118     3
67      3
126     3
59      3
134     3
51      3
142     3
14      3
179     3
6       3
177     3
31      3
23      3
15      3
7       3
201     3
193     3
185     3
169     3
47      3
161     3
153     3
145     3
137     3
129     3
121     3
113     3
39      3


In [9]:
data['user_id'].value_counts()

b3f7233aa1812ba3241595ba24623504    12
a1867661189878cbc0f6e2a3139b5dd5    11
371370ec7e273ec0a12ad91601a877b6    11
b49e0964e2ee594a380ff8f702df3e0b    10
4391ca69eb0e93c6752125f785fa8ef6    10
f789722b27212dc24eeb95e2af9849a5     9
67e05e3f315b6debdb7cdd590468bc6b     9
bb7ac8579bacf5bb769912383af88c18     9
c263c79c536e629184b0f1af53518d9b     8
648fd33febe8bf5da0e7709440155f5c     8
33fad46281dcb56a78a6e2a8a75473de     8
ae97084af50052b1a17d6fc7d2402b5b     8
af78e05a5f1aa2871a001657c2a9628e     8
2a22741f0d1ef41629508f22214df889     8
e0c86b7daa9969202e163ab7063e6d0b     8
6ff96e3d66dd8fe6980f49663f54dddb     8
ad38f80c435e7eba25a0d61c170a4e43     8
47b4ff9fc40be393ccd06dcba3e0a118     8
27591f06a9b43ef55b5e829a0ad8e494     8
ee367902b6c9ed2708b5d6fae8500bec     7
3e8b6598df12e8d090a756cd9db6a76e     7
3e56377de89ee0a0480525f9336a688d     7
a03006f5180693fec9a549bda99f6905     7
40e3025d15e1dadfd5b8bd241675433f     7
6c41989b0aabbd7f546812d856055ba4     7
8f7538646aba293fb25ce3009

In [20]:
data[data['ip_address_int'] == 3286945894]

,ID,user_id,ip_address_int,click_timestamp,source,sub_source,sub_source_2,sub_source_3,referer,fingerprint,ua
179,59,d5f768a79c12c89f01d32b817676da2c,3286945894,1635754866,250820642ba26fc02d946cf504440fdf,fba462c8894f1111b7efb1c67a094cf3,7791ccbd2270c2d91531262715bad131,91cc61b3a85e20194659c63d83a68cec,fba462c8894f1111b7efb1c67a094cf3,8306b77e0209cfff259fedf7ed16bc2f,axios/0.22.0
181,60,d5f768a79c12c89f01d32b817676da2c,3286945894,1635754866,250820642ba26fc02d946cf504440fdf,fba462c8894f1111b7efb1c67a094cf3,7791ccbd2270c2d91531262715bad131,91cc61b3a85e20194659c63d83a68cec,fba462c8894f1111b7efb1c67a094cf3,8306b77e0209cfff259fedf7ed16bc2f,axios/0.22.0
184,61,d5f768a79c12c89f01d32b817676da2c,3286945894,1635754866,250820642ba26fc02d946cf504440fdf,fba462c8894f1111b7efb1c67a094cf3,7791ccbd2270c2d91531262715bad131,91cc61b3a85e20194659c63d83a68cec,fba462c8894f1111b7efb1c67a094cf3,8306b77e0209cfff259fedf7ed16bc2f,axios/0.22.0
188,62,d5f768a79c12c89f01d32b817676da2c,3286945894,1635754866,250820642ba26fc02d946cf504440fdf,fba462c8894f1111b7efb1c67a094cf3,7791ccbd2270c2d91531262715bad131,91cc61b3a85e20194659c63d83a68cec,fba462c8894f1111b7efb1c67a094cf3,8306b77e0209cfff259fedf7ed16bc2f,axios/0.22.0
190,63,d5f768a79c12c89f01d32b817676da2c,3286945894,1635754866,250820642ba26fc02d946cf504440fdf,fba462c8894f1111b7efb1c67a094cf3,7791ccbd2270c2d91531262715bad131,91cc61b3a85e20194659c63d83a68cec,fba462c8894f1111b7efb1c67a094cf3,8306b77e0209cfff259fedf7ed16bc2f,axios/0.22.0
192,64,d5f768a79c12c89f01d32b817676da2c,3286945894,1635754866,250820642ba26fc02d946cf504440fdf,fba462c8894f1111b7efb1c67a094cf3,7791ccbd2270c2d91531262715bad131,91cc61b3a85e20194659c63d83a68cec,fba462c8894f1111b7efb1c67a094cf3,8306b77e0209cfff259fedf7ed16bc2f,axios/0.22.0
495,165,0d7901d2626907822eb0cc6574c1b0df,3286945894,1635756144,250820642ba26fc02d946cf504440fdf,fba462c8894f1111b7efb1c67a094cf3,7791ccbd2270c2d91531262715bad131,91cc61b3a85e20194659c63d83a68cec,fba462c8894f1111b7efb1c67a094cf3,8306b77e0209cfff259fedf7ed16bc2f,axios/0.22.0
499,166,a921abe2db4156b95730bf239451bd3a,3286945894,1635756144,250820642ba26fc02d946cf504440fdf,fba462c8894f1111b7efb1c67a094cf3,7791ccbd2270c2d91531262715bad131,91cc61b3a85e20194659c63d83a68cec,fba462c8894f1111b7efb1c67a094cf3,8306b77e0209cfff259fedf7ed16bc2f,axios/0.22.0
502,167,08ddb2d341541fec669c2eb596fb9f8b,3286945894,1635756144,250820642ba26fc02d946cf504440fdf,fba462c8894f1111b7efb1c67a094cf3,7791ccbd2270c2d91531262715bad131,91cc61b3a85e20194659c63d83a68cec,fba462c8894f1111b7efb1c67a094cf3,8306b77e0209cfff259fedf7ed16bc2f,axios/0.22.0
504,168,d0368b87d506e146e279f768d6dc646d,3286945894,1635756144,250820642ba26fc02d946cf504440fdf,fba462c8894f1111b7efb1c67a094cf3,7791ccbd2270c2d91531262715bad131,91cc61b3a85e20194659c63d83a68cec,fba462c8894f1111b7efb1c67a094cf3,8306b77e0209cfff259fedf7ed16bc2f,axios/0.22.0


In [33]:
data['ip_address_int'].value_counts()
# убрать 127.0.0.1

3286945894    299
1297308922    287
1546059692    265
1438660769    253
3267270763    251
3267270729    244
1438660704    233
1438660631    212
2197793299    211
1297308699    196
772878325     193
772930238     180
1438660774    178
2197793290    174
2276846441     19
3260535400     17
1441325442      9
1541242508      9
1500086415      8
1348123413      8
1500087528      6
1500086470      6
1348123578      5
3253299260      5
1348123501      5
1348123293      5
1608335421      5
1348124132      5
2960508214      5
1539640294      4
1546856351      4
1348122261      4
3253299956      4
1348124436      4
3253297928      4
3253299282      4
1546850175      4
92909860        3
3253297514      3
1541175533      3
1845267348      3
1546848580      3
1598345009      3
1546850218      3
1546849702      3
1546850811      3
1546858214      3
1546855799      3
1348123748      3
1348124569      3
1500612816      3
3647700676      3
87592795        3
1442388860      3
1546856173      3
1546850225

In [32]:
import time
# time.strftime("%D %H:%M", time.localtime(int("1635732719")))
unix_timestamp  = int("1635779485")
utc_time = time.gmtime(unix_timestamp)
local_time = time.localtime(unix_timestamp)
print(time.strftime("%Y-%m-%d %H:%M:%S", local_time)) 
# 2021-11-01 05:11:59
# 2021-11-01 18:11:25

2021-11-01 18:11:25


In [25]:
data['click_timestamp'].value_counts()

1635732719    40
1635779485    39
1635724936    34
1635742166    34
1635742165    34
1635727887    33
1635718165    31
1635740882    31
1635779486    29
1635732718    29
1635740881    29
1635740885    29
1635779481    29
1635753966    27
1635717789    27
1635779483    27
1635768892    26
1635756142    26
1635740888    26
1635780765    25
1635742164    25
1635796142    25
1635771538    24
1635732715    24
1635718160    24
1635729914    24
1635768895    24
1635755243    23
1635720342    23
1635755245    23
1635771837    22
1635718169    22
1635756144    22
1635780766    22
1635718166    21
1635727885    21
1635793843    21
1635724932    21
1635756146    20
1635738082    20
1635787529    20
1635727888    20
1635751540    20
1635720349    19
1635731194    19
1635790915    19
1635732712    19
1635779487    19
1635756141    19
1635720345    19
1635754866    19
1635717782    19
1635714269    19
1635731192    18
1635755242    18
1635731191    18
1635740889    18
1635787528    18
1635717787    

In [11]:
data['source'].value_counts()

250820642ba26fc02d946cf504440fdf    1783
5d1b41825a5dbe6b588ab93c77bb9bc5    1412
aec15b6423f1ce2204c44e4c1c88a49d    1104
1e2d9bfeaa5294cb4c94bd0891c2a993     424
ae83c0a39e40e6b477212fe34e464e10     283
e565eb7d382e99da8819f554f7b79b73     199
21ff72002536967f7679bcd199357340     181
35c45e10b97fcd1a769aadf3cfe32dc3     123
b16ae914063900219728933067a895c5     111
16b75a5bb6cb5b33ea67fd79a8c92f02     109
5891bf6d436ff467a0a014450b024427      97
e709cf43047c24ae7f19442f65f3ae54      89
2936929714756ae59d1309da8b7b7012      87
de9b102a5001d8ee701613461e2c92a6      74
ba9800bb99c65d83e3eea2811d47d285      74
8b8e744771bdb7b03f9ce9b7a8d65d48      72
2d11143b41443339d0ec16ce83cdc5f9      66
c45f51c99c4a8cd45fb7cab40c4aec72      65
585f9f6018f072619f3d8f4ac2fdcef1      63
ac1141f1d48fd35b4827d5ebd43659d4      62
ce8e4b641cb79fa8b2ad12771cb7773b      57
925225ce12515f7c6407185859a12579      54
324b7ecf25a026abc9d19df0863abb66      53
9d882d78bf1156a8e7bd5e734305e914      44
8fb7d4a0f20d948a

In [16]:
data['sub_source'].value_counts()

fba462c8894f1111b7efb1c67a094cf3    8613
47b1779f6f0a59fe001fa75145b21af3      17
8a48849b84adc1ad64156311b5c7230f       3
Name: sub_source, dtype: int64

In [17]:
data['sub_source_2'].value_counts()

7791ccbd2270c2d91531262715bad131    3196
a1d1ec2365378c2e69e3c48417141bd0    1104
03eccacdba8c56eae7b62cdb3a37f478     514
7247f04c4587fa3987db77679c0c54f3     243
cb7f49418b382f7fcb53e65a36c88cd6     199
                                    ... 
26458f28da2b3a232b11f2cbb36e1d23       1
3cc62da1541fba16ff415e91703d0c1e       1
64bdbf7f596acb3cf6c6980d78121e3b       1
066f5d59314bb7b2b6b4ec1b6b90d023       1
2c5d4484ef6c2dbafff137d2bf04a9db       1
Name: sub_source_2, Length: 494, dtype: int64

In [18]:
data['sub_source_3'].value_counts()

fba462c8894f1111b7efb1c67a094cf3    5413
91cc61b3a85e20194659c63d83a68cec    3195
c261bb236d04b447ab8aa63b8bd20105      17
605c5037f1a08d6158442d203bcd9f5e       3
98dd946b80eaf323f9f6c3e9ec840744       1
715dc80a3204f821521ebd81a6df6d5e       1
5bc7cdb17068d6908ef5c316a6e18df1       1
c07df4b870039aaa6afa82583cbd1864       1
26576d4667d3f068c33c6d5683cd9d83       1
Name: sub_source_3, dtype: int64

In [19]:
data['referer'].value_counts()

fba462c8894f1111b7efb1c67a094cf3    3215
465d9bf58432d4e1f422b3639caddee8    1522
b4489dd7c3eca220cbd23ccfedefa52f    1073
ad38f80c435e7eba25a0d61c170a4e43     659
45a9e777db654a627e8acf739b6d7457     485
                                    ... 
d11829f1cd398e8f96f3ad2f3ac7436b       1
c7372400c66feb02effdb809dd16fd17       1
f16511b22799f461c55004b24ced796d       1
ab1342bd1ad00fadc6bd48b72a0a0c9a       1
fb9467bbf0a6ace8477a9acefc88ab62       1
Name: referer, Length: 193, dtype: int64

In [20]:
data['fingerprint'].value_counts()

66b6125631b826ac0f3b4caf308b25ad    2896
fba462c8894f1111b7efb1c67a094cf3    1677
190b30133a125ecf6d995ca35a3f1df7     389
8306b77e0209cfff259fedf7ed16bc2f     299
8a13e50be57991aa811f2fd984e6600e      49
                                    ... 
d48b0092f667c84bb8ae6668593d9a9c       1
94945af063b0721d2886a34974e9d9ce       1
ada5aebda28eea28ca218d7b4edf467f       1
6201afe0db1141f2b434bd59f1998658       1
06312d158a1c6aadf6ee3b05d2b378ba       1
Name: fingerprint, Length: 740, dtype: int64

In [35]:
data['ua'].value_counts()

axios/0.23.0                                                                                                                                                                                                                                                                                                   1940
axios/0.22.0                                                                                                                                                                                                                                                                                                   1059
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/95.0.4638.69 Safari/537.36                                                                                                                                                                                                197
axios/0.24.0                                                                

**ua(user agent)**: 

1. Android 8.1.0
Mozilla/5.0 (Linux; Android 8.1.0; BV9500Pro Build/O11019; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/91.0.4472.120 Mobile Safari/537.36 
2. Axios – открытая библиотека для поддержки HTTP-запросов.
3. HeadlessChrome - боты
4. AdBot - размещения рекламы в ботах adbot.io